In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import seaborn as sns
import anndata
import scanpy as sc
import genetools
from covid_serology import config, helpers
from numpy.testing import assert_array_equal

# Load data

In [2]:
vaccine_participants = pd.read_csv(
    f"{config.paths.data_dir}/pfizer_demographics.csv"
).dropna(how="all")
vaccine_participants

,PID,COVID Positive Ever?
0,pfizer00,No
1,pfizer01,No
2,pfizer02,No
3,pfizer03,No
4,pfizer04,No
5,pfizer05,No
6,pfizer06,No
7,pfizer07,No
8,pfizer08,No
9,pfizer09,No


In [3]:
vaccine_df = pd.read_csv(f"{config.paths.data_dir}/pfizer_p11_boost.csv")
vaccine_df

,participant,timepoint,Epsilon_ECL,Epsilon_AU,Beta_ECL,Beta_AU,Iota_ECL,Iota_AU,Gamma_ECL,Gamma_AU,...,Alpha_ECL,Alpha_AU,P.3_ECL,P.3_AU,Kappa_ECL,Kappa_AU,Delta_ECL,Delta_AU,Wuhan_ECL,Wuhan_AU
0,pfizer00,D0,3828,1017.034757,3192,1418.890256,3273,1095.909180,3539,861.203717,...,3736,915.731658,3299,1104.856408,4013,1131.699710,4017,811.480795,3779,954.841762
1,pfizer01,D0,522,113.485400,381,128.472408,479,127.293665,349,58.639990,...,425,81.016962,258,55.965723,516,116.340069,464,74.591103,581,124.307340
2,pfizer02,D0,412,84.060552,301,93.482610,376,93.242917,410,73.095575,...,395,73.670487,331,79.196105,415,88.286803,365,54.959538,488,100.904751
3,pfizer03,D0,497,106.787884,345,112.690197,469,123.974357,385,67.158179,...,402,75.383592,287,65.154067,537,122.197315,451,72.004871,541,114.228705
4,pfizer04,D0,583,129.848939,392,133.305522,447,116.681383,540,104.211471,...,500,99.430972,414,105.956261,581,134.493574,437,69.222333,668,146.288540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,pfizer56,boostD1/2,8635,5892.465854,3168,2120.916452,5415,3940.827746,4831,2994.041935,...,5971,3688.680554,3292,2419.424349,7143,5900.000081,8584,4565.699428,9193,6073.602207
240,pfizer57,boostD1/2,1433,938.392743,592,326.020256,1009,643.872038,757,409.771568,...,1020,571.777118,568,347.255339,1113,862.419928,1417,720.005216,1431,888.851983
241,pfizer58,boostD1/2,10405,7107.068865,2892,1923.301930,5387,3919.405017,5004,3105.440737,...,6598,4088.066259,3323,2443.507295,7984,6603.373993,10157,5408.961427,10566,6995.486658
242,pfizer59,boostD1/2,3573,2413.490746,1184,724.991395,1975,1348.432754,1915,1133.350791,...,2750,1650.496368,1256,858.800469,2752,2230.211687,3407,1788.710763,4128,2682.861959


In [4]:
# only one sample per patient per timepoint
assert all(vaccine_df.groupby(["participant", "timepoint"]).size() == 1)

# Reshape vaccine data

In [5]:
vaccine_df.columns

Index(['participant', 'timepoint', 'Epsilon_ECL', 'Epsilon_AU', 'Beta_ECL',
       'Beta_AU', 'Iota_ECL', 'Iota_AU', 'Gamma_ECL', 'Gamma_AU',
       'B.1.526.2_ECL', 'B.1.526.2_AU', 'Alpha_ECL', 'Alpha_AU', 'P.3_ECL',
       'P.3_AU', 'Kappa_ECL', 'Kappa_AU', 'Delta_ECL', 'Delta_AU', 'Wuhan_ECL',
       'Wuhan_AU'],
      dtype='object')

In [6]:
vaccine_df = pd.melt(
    vaccine_df, id_vars=["participant", "timepoint"], var_name="measurement"
)
vaccine_df

,participant,timepoint,measurement,value
0,pfizer00,D0,Epsilon_ECL,3828.000000
1,pfizer01,D0,Epsilon_ECL,522.000000
2,pfizer02,D0,Epsilon_ECL,412.000000
3,pfizer03,D0,Epsilon_ECL,497.000000
4,pfizer04,D0,Epsilon_ECL,583.000000
...,...,...,...,...
4875,pfizer56,boostD1/2,Wuhan_AU,6073.602207
4876,pfizer57,boostD1/2,Wuhan_AU,888.851983
4877,pfizer58,boostD1/2,Wuhan_AU,6995.486658
4878,pfizer59,boostD1/2,Wuhan_AU,2682.861959


In [7]:
vaccine_df = vaccine_df[vaccine_df["measurement"].str.contains("AU")].copy()
vaccine_df

,participant,timepoint,measurement,value
244,pfizer00,D0,Epsilon_AU,1017.034757
245,pfizer01,D0,Epsilon_AU,113.485400
246,pfizer02,D0,Epsilon_AU,84.060552
247,pfizer03,D0,Epsilon_AU,106.787884
248,pfizer04,D0,Epsilon_AU,129.848939
...,...,...,...,...
4875,pfizer56,boostD1/2,Wuhan_AU,6073.602207
4876,pfizer57,boostD1/2,Wuhan_AU,888.851983
4877,pfizer58,boostD1/2,Wuhan_AU,6995.486658
4878,pfizer59,boostD1/2,Wuhan_AU,2682.861959


In [8]:
# just in case, convert to float and switch non-numeric values to nan
vaccine_df["value"] = pd.to_numeric(vaccine_df["value"], errors="coerce")
vaccine_df.dtypes

participant     object
timepoint       object
measurement     object
value          float64
dtype: object

In [9]:
vaccine_df["value"].isna().value_counts()

False    2440
Name: value, dtype: int64

In [10]:
vaccine_df[vaccine_df["value"].isna()]

,participant,timepoint,measurement,value


In [11]:
measurement_cols = vaccine_df["measurement"].unique()
helpers.confirm_all_measurement_columns_are_present(measurement_cols)
measurement_cols

array(['Epsilon_AU', 'Beta_AU', 'Iota_AU', 'Gamma_AU', 'B.1.526.2_AU',
       'Alpha_AU', 'P.3_AU', 'Kappa_AU', 'Delta_AU', 'Wuhan_AU'],
      dtype=object)

In [12]:
vaccine_df["measurement"] = vaccine_df["measurement"].str.replace("_AU", "")
vaccine_df["measurement"].value_counts()

Gamma        244
Wuhan        244
Beta         244
P.3          244
Epsilon      244
Kappa        244
Iota         244
Alpha        244
B.1.526.2    244
Delta        244
Name: measurement, dtype: int64

In [13]:
# extract parts of measurement column
# all variant plate measurements are IgG only
measurement_parts = (
    vaccine_df["measurement"]
    .rename("virus")
    .to_frame()
    .assign(variant_plate_type="Variant", target="RBD", antibody="IgG")
    .apply(lambda col: col.str.strip())
)
measurement_parts

,virus,variant_plate_type,target,antibody
244,Epsilon,Variant,RBD,IgG
245,Epsilon,Variant,RBD,IgG
246,Epsilon,Variant,RBD,IgG
247,Epsilon,Variant,RBD,IgG
248,Epsilon,Variant,RBD,IgG
...,...,...,...,...
4875,Wuhan,Variant,RBD,IgG
4876,Wuhan,Variant,RBD,IgG
4877,Wuhan,Variant,RBD,IgG
4878,Wuhan,Variant,RBD,IgG


In [14]:
measurement_parts["virus"].value_counts()

Gamma        244
Wuhan        244
Beta         244
P.3          244
Epsilon      244
Kappa        244
Iota         244
Alpha        244
B.1.526.2    244
Delta        244
Name: virus, dtype: int64

In [15]:
measurement_parts["target"].value_counts()

RBD    2440
Name: target, dtype: int64

In [16]:
measurement_parts["variant_plate_type"].value_counts()

Variant    2440
Name: variant_plate_type, dtype: int64

In [17]:
vaccine_df = pd.concat([vaccine_df, measurement_parts], axis=1)
vaccine_df

,participant,timepoint,measurement,value,virus,variant_plate_type,target,antibody
244,pfizer00,D0,Epsilon,1017.034757,Epsilon,Variant,RBD,IgG
245,pfizer01,D0,Epsilon,113.485400,Epsilon,Variant,RBD,IgG
246,pfizer02,D0,Epsilon,84.060552,Epsilon,Variant,RBD,IgG
247,pfizer03,D0,Epsilon,106.787884,Epsilon,Variant,RBD,IgG
248,pfizer04,D0,Epsilon,129.848939,Epsilon,Variant,RBD,IgG
...,...,...,...,...,...,...,...,...
4875,pfizer56,boostD1/2,Wuhan,6073.602207,Wuhan,Variant,RBD,IgG
4876,pfizer57,boostD1/2,Wuhan,888.851983,Wuhan,Variant,RBD,IgG
4877,pfizer58,boostD1/2,Wuhan,6995.486658,Wuhan,Variant,RBD,IgG
4878,pfizer59,boostD1/2,Wuhan,2682.861959,Wuhan,Variant,RBD,IgG


In [18]:
vaccine_df["timepoint"].value_counts()

D21          450
D28          440
D0           430
D90          370
D210         350
boostD1/2    220
boostD7/8    150
boostD21      30
Name: timepoint, dtype: int64

In [19]:
# timepoint label map
map_vaccine_to_global_timepoint_labels = {
    "D0": "day 0 / pre-pandemic",
    "D7": "day 7 / week 1",
    "D21": "day 21 / weeks 2&3",
    "D28": "day 28 / week 4",
    "D42": "day 42 / weeks 5&6",
    "D90": "week 7 and later / 3 months",
    "D210": "day 210 / 7 months",
    "boostD1/2": "boostD1/2",
    "boostD7/8": "boostD7/8",
    "boostD21": "boostD21",
}
assert all(
    tp in map_vaccine_to_global_timepoint_labels.keys()
    for tp in vaccine_df["timepoint"].unique()
)

In [20]:
def process_vaccine_timepoint(df_partial, timepoint):
    # at a given time point: only one measurement per patient-virus-target combo
    assert all(
        df_partial.groupby(
            ["participant", "virus", "target", "variant_plate_type", "antibody"]
        ).size()
        == 1
    )

    # unmelt
    vaccine_df_pivot = pd.pivot(
        df_partial,
        index="participant",
        columns=["virus", "target", "variant_plate_type", "antibody"],
        values="value",
    )

    ## set column names
    variable_info = vaccine_df_pivot.columns.to_frame().reset_index(drop=True)
    # create combined name
    variable_info["timepoint"] = timepoint
    variable_info["combined_name"] = variable_info.apply("_".join, axis=1)
    variable_info = variable_info.set_index("combined_name")

    # set var names
    vaccine_df_pivot.columns = variable_info.index.copy()

    # drop patients with any NaNs in this timepoint
    vaccine_df_pivot = vaccine_df_pivot.dropna(how="any")
    assert not vaccine_df_pivot.isna().any().any()

    return vaccine_df_pivot, variable_info

In [21]:
X_partial = []
var_partial = []
for vaccine_timepoint in vaccine_df["timepoint"].unique():
    associated_global_timepoint_label = map_vaccine_to_global_timepoint_labels[
        vaccine_timepoint
    ]
    print(vaccine_timepoint, "->", associated_global_timepoint_label)
    df_partial = vaccine_df[vaccine_df["timepoint"] == vaccine_timepoint]
    vaccine_df_pivot, variable_info = process_vaccine_timepoint(
        df_partial, associated_global_timepoint_label
    )
    X_partial.append(vaccine_df_pivot)
    var_partial.append(variable_info)
vaccine_df_pivot = pd.concat(X_partial, axis=1)
variable_info = pd.concat(var_partial, axis=0)

D0 -> day 0 / pre-pandemic
D21 -> day 21 / weeks 2&3
D28 -> day 28 / week 4
D90 -> week 7 and later / 3 months
D210 -> day 210 / 7 months
boostD1/2 -> boostD1/2
boostD7/8 -> boostD7/8
boostD21 -> boostD21


In [22]:
# note: there are NaNs - patients don't have entries for all timepoints
vaccine_df_pivot

combined_name,Epsilon_RBD_Variant_IgG_day 0 / pre-pandemic,Beta_RBD_Variant_IgG_day 0 / pre-pandemic,Iota_RBD_Variant_IgG_day 0 / pre-pandemic,Gamma_RBD_Variant_IgG_day 0 / pre-pandemic,B.1.526.2_RBD_Variant_IgG_day 0 / pre-pandemic,Alpha_RBD_Variant_IgG_day 0 / pre-pandemic,P.3_RBD_Variant_IgG_day 0 / pre-pandemic,Kappa_RBD_Variant_IgG_day 0 / pre-pandemic,Delta_RBD_Variant_IgG_day 0 / pre-pandemic,Wuhan_RBD_Variant_IgG_day 0 / pre-pandemic,...,Epsilon_RBD_Variant_IgG_boostD21,Beta_RBD_Variant_IgG_boostD21,Iota_RBD_Variant_IgG_boostD21,Gamma_RBD_Variant_IgG_boostD21,B.1.526.2_RBD_Variant_IgG_boostD21,Alpha_RBD_Variant_IgG_boostD21,P.3_RBD_Variant_IgG_boostD21,Kappa_RBD_Variant_IgG_boostD21,Delta_RBD_Variant_IgG_boostD21,Wuhan_RBD_Variant_IgG_boostD21
participant,,,,,,,,,,,,,,,,,,,,,
pfizer00,1017.034757,1418.890256,1095.909180,861.203717,862.666390,915.731658,1104.856408,1131.699710,811.480795,954.841762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer01,113.485400,128.472408,127.293665,58.639990,83.940808,81.016962,55.965723,116.340069,74.591103,124.307340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer02,84.060552,93.482610,93.242917,73.095575,71.918525,73.670487,79.196105,88.286803,54.959538,100.904751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer03,106.787884,112.690197,123.974357,67.158179,75.089555,75.383592,65.154067,122.197315,72.004871,114.228705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer04,129.848939,133.305522,116.681383,104.211471,100.799579,99.430972,105.956261,134.493574,69.222333,146.288540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer05,95.816259,70.489811,84.373942,56.752554,49.831210,62.430578,57.861961,71.729805,58.517283,86.618318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer06,113.485400,108.316480,125.965625,86.931707,66.942754,70.735323,85.935731,78.893216,51.801985,107.436514,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer07,108.394789,110.502755,98.840305,78.095743,161.704607,169.126599,70.565899,83.033966,68.030678,187.405854,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pfizer08,118.310900,112.252616,138.931366,79.526393,77.810314,84.204026,109.200940,94.379438,60.694187,127.082166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
variable_info

,virus,target,variant_plate_type,antibody,timepoint
combined_name,,,,,
Epsilon_RBD_Variant_IgG_day 0 / pre-pandemic,Epsilon,RBD,Variant,IgG,day 0 / pre-pandemic
Beta_RBD_Variant_IgG_day 0 / pre-pandemic,Beta,RBD,Variant,IgG,day 0 / pre-pandemic
Iota_RBD_Variant_IgG_day 0 / pre-pandemic,Iota,RBD,Variant,IgG,day 0 / pre-pandemic
Gamma_RBD_Variant_IgG_day 0 / pre-pandemic,Gamma,RBD,Variant,IgG,day 0 / pre-pandemic
B.1.526.2_RBD_Variant_IgG_day 0 / pre-pandemic,B.1.526.2,RBD,Variant,IgG,day 0 / pre-pandemic
...,...,...,...,...,...
Alpha_RBD_Variant_IgG_boostD21,Alpha,RBD,Variant,IgG,boostD21
P.3_RBD_Variant_IgG_boostD21,P.3,RBD,Variant,IgG,boostD21
Kappa_RBD_Variant_IgG_boostD21,Kappa,RBD,Variant,IgG,boostD21


In [24]:
# attach status

vaccine_participants = vaccine_participants.set_index("PID")
vaccine_participants["Status"] = "Vaccinee"
# reorder
vaccine_participants = vaccine_participants.loc[vaccine_df_pivot.index]

# confirm same order
assert_array_equal(vaccine_participants.index, vaccine_df_pivot.index)

vaccine_participants

,COVID Positive Ever?,Status
participant,,
pfizer00,No,Vaccinee
pfizer01,No,Vaccinee
pfizer02,No,Vaccinee
pfizer03,No,Vaccinee
pfizer04,No,Vaccinee
pfizer05,No,Vaccinee
pfizer06,No,Vaccinee
pfizer07,No,Vaccinee
pfizer08,No,Vaccinee


In [25]:
# anndata requires string indices
vaccine_participants.index = vaccine_participants.index.astype(str)
vaccine_df_pivot.index = vaccine_df_pivot.index.astype(str)

In [26]:
adata_vaccine = anndata.AnnData(
    X=vaccine_df_pivot, obs=vaccine_participants, var=variable_info
)
adata_vaccine

AnnData object with n_obs × n_vars = 53 × 80
    obs: 'COVID Positive Ever?', 'Status'
    var: 'virus', 'target', 'variant_plate_type', 'antibody', 'timepoint'

In [27]:
adata_vaccine.var

,virus,target,variant_plate_type,antibody,timepoint
combined_name,,,,,
Epsilon_RBD_Variant_IgG_day 0 / pre-pandemic,Epsilon,RBD,Variant,IgG,day 0 / pre-pandemic
Beta_RBD_Variant_IgG_day 0 / pre-pandemic,Beta,RBD,Variant,IgG,day 0 / pre-pandemic
Iota_RBD_Variant_IgG_day 0 / pre-pandemic,Iota,RBD,Variant,IgG,day 0 / pre-pandemic
Gamma_RBD_Variant_IgG_day 0 / pre-pandemic,Gamma,RBD,Variant,IgG,day 0 / pre-pandemic
B.1.526.2_RBD_Variant_IgG_day 0 / pre-pandemic,B.1.526.2,RBD,Variant,IgG,day 0 / pre-pandemic
...,...,...,...,...,...
Alpha_RBD_Variant_IgG_boostD21,Alpha,RBD,Variant,IgG,boostD21
P.3_RBD_Variant_IgG_boostD21,P.3,RBD,Variant,IgG,boostD21
Kappa_RBD_Variant_IgG_boostD21,Kappa,RBD,Variant,IgG,boostD21


In [28]:
adata_vaccine.obs["Status"].value_counts()

Vaccinee    53
Name: Status, dtype: int64

In [29]:
adata_vaccine.write(
    f"{config.paths.generated_data_dir}/partial.pfizer_vaccine.variant_plate.h5"
)

... storing 'COVID Positive Ever?' as categorical


... storing 'Status' as categorical


... storing 'virus' as categorical


... storing 'target' as categorical


... storing 'variant_plate_type' as categorical


... storing 'antibody' as categorical


... storing 'timepoint' as categorical
